In [49]:
import pandas as pd
import gensim
from gensim.models.phrases import Phrases, Phraser, ENGLISH_CONNECTOR_WORDS
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tqdm import tqdm
import numpy as np
import enchant
from time import time 
import multiprocessing
from copy import deepcopy
from collections import defaultdict

In [3]:
tokens = np.load('tokens_per_year.npy',allow_pickle='FALSE').item()

In [13]:
#documents è la lista di liste
#doc sono le liste interne
def build_bigram_dictionary(documents, min_count, threshold):
    phrases = Phrases(documents, min_count=min_count, threshold=threshold, connector_words=ENGLISH_CONNECTOR_WORDS)
    bigram = Phraser(phrases)
    tokens_text_with_bigrams = [bigram[doc] for doc in documents]
    return tokens_text_with_bigrams

**TRAIN WORD2VEC MODEL ON THE WHOLE SET**.

In [22]:
# CREATE A VOCABULARY OF BIGRAMS THE DATASET PER YEAR
bigram_vocab = {}
for year, tokens in tokens.items():
    tokens_per_year = build_bigram_dictionary(tokens, 11, 20)
    bigram_vocab[year] = tokens_per_year

In [26]:
bigram_vocab['2006'][0]

['municipal',
 'waste',
 'country',
 'increase',
 'around',
 'paper',
 'outline',
 'real',
 'negative',
 'effect',
 'increase',
 'introduce',
 'two',
 'main',
 'euro_pean',
 'union',
 'policy',
 'establish',
 'address',
 'problem',
 'landfill',
 'directive',
 'legislation',
 'extend',
 'producer',
 'responsibility',
 'paper',
 'describe',
 'compare',
 'four',
 'alternative',
 'strategy',
 'reducing',
 'end',
 'life',
 'waste',
 'within',
 'context',
 'extend',
 'producer',
 'responsibility',
 'namely',
 'repair',
 'recondition',
 'remanufacture',
 'recycle',
 'also',
 'introduce',
 'robust',
 'definition',
 'rem',
 'fact',
 'ring',
 'validate',
 'early',
 'research',
 'differentiate',
 'repair',
 'reconditioning',
 'consideration',
 'different',
 'factor',
 'involve',
 'conclude',
 'rem',
 'fact',
 'ring',
 'may',
 'well',
 'good',
 'strategy',
 'enable',
 'embody',
 'energy',
 'virgin',
 'production',
 'maintain',
 'preserve',
 'intrinsic',
 'add',
 'value',
 'product',
 'manufacturer

In [27]:
wholeset =  [doc for papers in bigram_vocab.values() for doc in papers]

In [33]:
word_freq = defaultdict(int)
for big in wholeset:
    for i in big:
        word_freq[i] += 1
len(word_freq)

20901

In [37]:
sorted(word_freq, key=word_freq.get, reverse=True)[:200]

['circular_economy',
 'use',
 'product',
 'waste',
 'material',
 'resource',
 'system',
 'environmental',
 'economic',
 'development',
 'also',
 'production',
 'energy',
 'economy',
 'process',
 'design',
 'new',
 'value',
 'model',
 'research',
 'recycling',
 'level',
 'business',
 'industry',
 'policy',
 'include',
 'base',
 'industrial',
 'strategy',
 'circular',
 'increase',
 'study',
 'need',
 'one',
 'management',
 'reduce',
 'recycle',
 'approach',
 'consumption',
 'technology',
 'business_model',
 'indicator',
 'concept',
 'different',
 'company',
 'analysis',
 'develop',
 'china',
 'well',
 'sustainability',
 'reuse',
 'make',
 'provide',
 'environment',
 'high',
 'sustainable',
 'example',
 'practice',
 'social',
 'may',
 'result',
 'change',
 'framework',
 'paper',
 'supply_chain',
 'cost',
 'service',
 'focus',
 'however',
 'support',
 'impact',
 'efficiency',
 'within',
 'plastic',
 'could',
 'emission',
 'activity',
 'flow',
 'potential',
 'show',
 'water',
 'require',
 '

In [41]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [42]:
#In this first step, we set up the parameters of the model one-by-one. we don't supply the parameter sentences, and therefore leave the model uninitialized, purposefully.
w2v_model = Word2Vec(
                     vector_size = 150,
                     min_count=10, #Ignores all words with total absolute frequency lower than this - (2, 100)                    
                     window=10,  #The maximum distance between the current and predicted word within a sentence. E.g. window words on the left and window words on the right of our target
                     sample=6e-5,   #Dimensionality of the feature vectors
                     alpha=0.03,  #The initial learning rate
                     min_alpha=0.0007,  #Learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00
                     #negative=10,  #If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
                     workers=cores-1  #Use these many worker threads to train the model (=faster training with multicore machines)
                     )

In [43]:
t = time()
#Here it builds the vocabulary from a sequence of sentences and thus initialized the model. With the loggings, We can follow the progress and even more important, the effect of min_count and sample on the word corpus. We noticed that these two parameters, and in particular sample, have a great influence over the performance of a model. Displaying both allows for a more accurate and an easier management of their influence.
#Word2Vec requires us to build the vocabulary table (simply digesting all the words and filtering out the unique words, and doing some basic counts on them)

w2v_model.build_vocab(wholeset)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.01 mins


In [45]:
t = time()
w2v_model.train(wholeset, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.28 mins


In [47]:
# salva il modello
model = w2v_model.save("word2vec_cb2.model")

In [50]:
ft_model = deepcopy(model)

**FINETUNING OF THE MODEL ON THE DATASET OF EACH YEAR**

In [54]:
model = Word2Vec.load("word2vec_cb2.model")

ft_models = {}
for year, documents in bigram_vocab.items():
    
    # Upload the deepcopy of the existing model each time the loop is restarted (for each year)
    ft_model = deepcopy(model)
    
    # Update the dictionary with the bigrams of the year and train the new model
    ft_model.build_vocab(documents, update=True)
    ft_model.train(documents, total_examples=ft_model.corpus_count, epochs=30)
    
    # Save the model
    ft_model.save(f"word2vec_{year}.model")
    ft_models[year] = ft_model

In [55]:
# SAVE THE VECTORS:
ft_vectors = {}
for year, model in ft_models.items():
    word_vectors = model.wv  
    ft_vectors[year] = word_vectors

In [56]:
for year, word_vector in ft_vectors.items():
    print(word_vector['circular_economy'].shape)

(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)
(150,)


In [58]:
import numpy as np
from sklearn.decomposition import PCA
import plotly.express as px

vectors = np.array([word_vector['circular_economy'] for year, word_vector in ft_vectors.items()])
years = list(ft_vectors.keys())

pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(vectors)

import pandas as pd
df = pd.DataFrame({
    'PCA1': reduced_vectors[:, 0],
    'PCA2': reduced_vectors[:, 1],
    'Year': years
})

fig = px.scatter(df, x='PCA1', y='PCA2', text='Year', title='2D Representation of Circular Economy embeddings"',
                 color='Year', template="plotly_white")
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=True)
fig.show()